Aggregation in SQL
-----

In [1]:
%load_ext sql
%sql sqlite:///world-db

Let's ask some SQL queries over the `world` dataset using aggregation. 
Our first query finds the average population of countries in Europe.

In [3]:
%%sql
SELECT AVG(Population), Name
FROM Country
WHERE Continent = 'Europe';

 * sqlite:///world-db
Done.


AVG(Population),Name
15871186.956521738,Albania


The next query outputs the number of countries in Europe.

In [4]:
%%sql
SELECT COUNT(*) 
FROM Country
WHERE Continent = 'Europe';

 * sqlite:///world-db
Done.


COUNT(*)
46


Next, we want to compute the number of languages in the `CountryLanguage` table. Here we want to use `DISTINCT` inside the count, otherwise languages that are spoken in multiple countries will be counted multiple times!

In [5]:
%%sql
SELECT COUNT(DISTINCT Language)
FROM CountryLanguage ;

 * sqlite:///world-db
Done.


COUNT(DISTINCT Language)
457


How can we find the name *and* population of the country with the max population in Europe? Notice that finding only the max population is easy, but if we want to output the country as well, it gets a bit tricky. We can write a nested query to solve this problem in an elegant way!

In [6]:
%%sql
SELECT Name, Population
FROM Country
WHERE Population = 
  (SELECT MAX(Population)
   FROM Country
   WHERE Continent = 'Europe');

 * sqlite:///world-db
Done.


Name,Population
Russian Federation,146934000


An alternative way is to use together `ORDER BY` and `LIMIT`. However, the two queries can sometimes give a different result. Can you think when this may happen?

> When two Countries have the same max value, only one will be selected


In [7]:
%%sql
SELECT Name, Population
FROM Country
WHERE Continent = 'Europe'
ORDER BY Population DESC
LIMIT 1 ;

 * sqlite:///world-db
Done.


Name,Population
Russian Federation,146934000


We will see next how to use `GROUP BY`. The following SQL query computes the number of countries in every continent.

In [11]:
%%sql
SELECT Continent, SUM(Population), COUNT(*) 
FROM Country 
GROUP BY Continent

 * sqlite:///world-db
Done.


Continent,SUM(Population),COUNT(*)
Africa,784475000,58
Antarctica,0,5
Asia,3705025700,51
Europe,730074600,46
North America,482993000,37
Oceania,30401150,28
South America,345780000,14


We can combine `GROUP BY` with `ORDER BY` as well. The following SQL query finds out how many countries speak each language with percentage > 50% in decreasing order.

In [ ]:
%%sql
SELECT Language, COUNT(CountryCode) as N
FROM CountryLanguage
WHERE Percentage >= 50
GROUP BY Language
ORDER BY N DESC ;

The `HAVING` clause allows us to express conditions over properties of *groups*, and not only individual tuples. `HAVING` always follows a `GROUP BY`. As an example, the following SQL query finds out the languages that are spoken in at least 3 different countries with percentage at least 50.

In [21]:
%%sql
SELECT *
FROM CountryLanguage
LIMIT 5

 * sqlite:///world-db
Done.


CountryCode,Language,IsOfficial,Percentage
ABW,Dutch,T,5.3
ABW,English,F,9.5
ABW,Papiamento,F,76.7
ABW,Spanish,F,7.4
AFG,Balochi,F,0.9


In [41]:
%%sql
SELECT Language, COUNT(CountryCode) as N
FROM CountryLanguage
WHERE Percentage >= 50 
GROUP BY Language
HAVING N > 2 

 * sqlite:///world-db
Done.


Language,N
Arabic,16
Creole English,8
Creole French,6
English,11
German,4
Serbo-Croatian,3
Spanish,20


## Example of a Country

In [42]:
%%sql 
SELECT *
FROM Country
Limit 1

 * sqlite:///world-db
Done.


Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
ABW,Aruba,North America,Caribbean,193.0,None,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW


## Example of a City

In [28]:
%%sql 
SELECT *
FROM City
Limit 1

 * sqlite:///world-db
Done.


ID,Name,CountryCode,District,Population
1,Kabul,AFG,Kabol,1780000


**Exercise #1**: Write a query that outputs for each country the population of the most populated city, for countries with at least 10 cities.

In [61]:
%%sql
SELECT C.Name, MAX(T.Population) as N
FROM City T, Country C
WHERE T.CountryCode = C.Code
GROUP BY C.Name
HAVING COUNT(T.ID) >= 100
ORDER BY N DESC

 * sqlite:///world-db
Done.


Name,N
India,10500000
Brazil,9968485
China,9696300
Mexico,8591309
Russian Federation,8389200
United States,8008278
Japan,7980230
Philippines,2173831


Let's see how the use of `HAVING` compares with the use of correlated queries. Suppose that we want to find the names of the countries that have more than 10 cities with population at least 1 million. Here is a nested query that computes that: 

In [22]:
%%sql
SELECT C.name
FROM Country C
WHERE (SELECT COUNT(*) 
       FROM City
       WHERE City.CountryCode=C.Code
       AND City.Population >= 1000000) > 10; 

 * sqlite:///world-db
Done.


Name
Brazil
China
India
Japan
Russian Federation


We can measure the execution time of the query using `%time`.

In [ ]:
%time %sql SELECT C.name FROM Country C WHERE (SELECT COUNT(*) FROM City WHERE City.CountryCode=C.Code AND City.Population >= 1000000) > 10; 

**Exercise #2**: Write the above query using `HAVING` and time its execution. How much faster does it run?